In [1]:
import pandas as pd
import requests
import json
import grequests

C:\Users\cello\Anaconda3\lib\site-packages\grequests.py:22: MonkeyPatchWarning: Monkey-patching ssl after ssl has already been imported may lead to errors, including RecursionError on Python 3.6. It may also silently lead to incorrect behaviour on Python 3.7. Please monkey-patch earlier. See https://github.com/gevent/gevent/issues/1016. Modules that had direct imports (NOT patched): ['requests.packages.urllib3.util (C:\\Users\\cello\\Anaconda3\\lib\\site-packages\\requests\\packages\\urllib3\\util\\__init__.py)', 'urllib3.contrib.pyopenssl (C:\\Users\\cello\\Anaconda3\\lib\\site-packages\\urllib3\\contrib\\pyopenssl.py)', 'requests.packages.urllib3.util.ssl_ (C:\\Users\\cello\\Anaconda3\\lib\\site-packages\\requests\\packages\\urllib3\\util\\ssl_.py)']. 
  curious_george.patch_all(thread=False, select=False)
C:\Users\cello\Anaconda3\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(se

In [2]:
raw = requests.get('https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=vaccination_data')
json = raw.json()['vaccination_data']
vaccinationData = pd.DataFrame(json)

In [3]:
vaccinationData

,Date,MMWR_week,Location,ShortName,LongName,Doses_Distributed,Doses_Administered,Dist_Per_100K,Admin_Per_100K,Census2019
0,2021-01-02,53,AK,AKA,Alaska,68925,18401,9422,2515,731545
1,2021-01-02,53,AL,ALA,Alabama,124275,39446,2535,804,4903185
2,2021-01-02,53,AR,ARA,Arkansas,124325,39235,4120,1300,3017804
3,2021-01-02,53,AS,ASA,American Samoa,8250,1022,14814,1835,55689
4,2021-01-02,53,AZ,AZA,Arizona,270250,59099,3713,812,7278717
...,...,...,...,...,...,...,...,...,...,...
60,2021-01-02,53,VT,VTA,Vermont,30025,15478,4812,2480,623989
61,2021-01-02,53,WA,WAA,Washington,282950,82464,3716,1083,7614893
62,2021-01-02,53,WI,WIA,Wisconsin,159800,64657,2745,1110,5822434
63,2021-01-02,53,WV,WVA,West Virginia,103375,49827,5768,2780,1792147


C:\Users\cello\Anaconda3\lib\site-packages\gevent\hub.py:161: UserWarning: libuv only supports millisecond timer resolution; all times less will be set to 1 ms
  with loop.timer(seconds, ref=ref) as t:


In [25]:
raw = requests.get('https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=integrated_county_timeseries_state_IL_external')
json = raw.json()['integrated_county_timeseries_external_data']
timeSeriesData = pd.DataFrame(json).sort_values('date')

In [31]:
filtered = timeSeriesData[['fips_code','date','report_date_window','report_date_window_start','percent_new_test_results_reported_positive_7_day_rolling_average','new_test_results_reported_7_day_rolling_average']]

In [32]:
filtered.head()

,fips_code,date,report_date_window,report_date_window_start,percent_new_test_results_reported_positive_7_day_rolling_average,new_test_results_reported_7_day_rolling_average
22224,17129,2020-01-22T00:00:00,2020-01-22T00:00:00,2020-01-16T00:00:00,0.0,0.0
19448,17119,2020-01-22T00:00:00,2020-01-22T00:00:00,2020-01-16T00:00:00,0.0,0.0
17723,17103,2020-01-22T00:00:00,2020-01-22T00:00:00,2020-01-16T00:00:00,0.0,0.0
33705,17195,2020-01-22T00:00:00,2020-01-22T00:00:00,2020-01-16T00:00:00,0.0,0.0
11512,17067,2020-01-22T00:00:00,2020-01-22T00:00:00,2020-01-16T00:00:00,0.0,0.0


In [36]:
filtered[filtered.fips_code == 17129][['date','percent_new_test_results_reported_positive_7_day_rolling_average']] \
    .set_index('date').T

date,2020-01-22T00:00:00,2020-01-23T00:00:00,2020-01-24T00:00:00,2020-01-25T00:00:00,2020-01-26T00:00:00,2020-01-27T00:00:00,2020-01-28T00:00:00,2020-01-29T00:00:00,2020-01-30T00:00:00,2020-01-31T00:00:00,...,2020-12-23T00:00:00,2020-12-24T00:00:00,2020-12-25T00:00:00,2020-12-26T00:00:00,2020-12-27T00:00:00,2020-12-28T00:00:00,2020-12-29T00:00:00,2020-12-30T00:00:00,2020-12-31T00:00:00,2021-01-01T00:00:00
percent_new_test_results_reported_positive_7_day_rolling_average,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5.42,5.99,6.34,6.78,7.35,8.37,8.3,8.22,10.08,11.24


In [61]:
state2Digit = ['AK', 'AL', 'AR', 'AZ', 'CA', 'CO', 'CT', 'DC', 'DE', 'FL', 'GA', 'HI', 'IA', 'ID', 'IL', 'IN', 'KS', 'KY', 'LA', 'MA', 'MD', 'ME', 'MI', 'MN', 'MO', 'MS', 'MT', 'NC', 'ND', 'NE', 'NH', 'NJ', 'NM', 'NV', 'NY', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VA', 'VT', 'WA', 'WI', 'WV', 'WY']

In [62]:
urls = [f"https://covid.cdc.gov/covid-data-tracker/COVIDData/getAjaxData?id=integrated_county_timeseries_state_{stateCode}_external" for stateCode in state2Digit]
breakpoint = 12
urlDict = []
for i in range(0,5):
    urlDict.append(urls[breakpoint*i:breakpoint*(i+1)])

In [63]:
responses = []
for urlList in urlDict:
    rs = (grequests.get(u,  timeout=120) for u in urlList)
    response = grequests.map(rs)
    responses.append(response)

In [69]:
parsed = ''

for responseSet in responses:
    for response in responseSet:
        if len(parsed)==0:
            parsed = pd.DataFrame(response.json()['integrated_county_timeseries_external_data'])
        else:
            parsed = pd.concat([parsed, pd.DataFrame(response.json()['integrated_county_timeseries_external_data'])])

parsed = parsed.sort_values('date')

In [106]:
uniqFips = list(parsed.fips_code.unique())

cleaned = ''

for fips in uniqFips:
    if len(cleaned) == 0:
        cleaned = parsed[parsed.fips_code == fips][['date','percent_new_test_results_reported_positive_7_day_rolling_average']] \
            .set_index('date').T
        cleaned['fips'] = fips
    else:
        tempDf = parsed[parsed.fips_code == fips][['date','percent_new_test_results_reported_positive_7_day_rolling_average']] \
            .set_index('date').T
        tempDf['fips'] = fips
        cleaned = pd.concat([cleaned, tempDf])
cleaned = cleaned[list(cleaned.columns)[-1:] + list(cleaned.columns)[:-1]]

In [110]:
columnNames = [col[:10] for col in cleaned.columns]

In [112]:
cleaned.columns = columnNames

In [115]:
cleaned.to_csv("./county_testing_data.csv",index=False)